In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 108.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [ ]:
!pip install gradio playwright pytesseract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
!playwright install

163.5 MiB [] 0% 0.0s163.5 MiB [] 0% 3.0s163.5 MiB [] 1% 2.0s163.5 MiB [] 2% 1.8s163.5 MiB [] 3% 1.6s163.5 MiB [] 4% 1.5s163.5 MiB [] 6% 1.5s163.5 MiB [] 6% 1.6s163.5 MiB [] 7% 1.6s163.5 MiB [] 8% 1.6s163.5 MiB [] 9% 1.5s163.5 MiB [] 10% 1.5s163.5 MiB [] 11% 1.5s163.5 MiB [] 13% 1.4s163.5 MiB [] 14% 1.4s163.5 MiB [] 15% 1.3s163.5 MiB [] 16% 1.3s163.5 MiB [] 17% 1.3s163.5 MiB [] 18% 1.3s163.5 MiB [] 19% 1.2s163.5 MiB [] 21% 1.2s163.5 MiB [] 22% 1.2s163.5 MiB [] 23% 1.1s163.5 MiB [] 24% 1.1s163.5 MiB [] 26% 1.1s163.5 MiB [] 27% 1.0s163.5 MiB [] 28% 1.1s163.5 MiB [] 29% 1.1s163.5 MiB [] 31% 1.0s163.5 MiB [] 33% 1.0s163.5 MiB [] 34% 0.9s163.5 MiB [] 36% 0.9s163.5 MiB [] 38% 0.8s163.5 MiB [] 39% 0.8s163.5 MiB [] 40% 0.8s163.5 MiB [] 41% 0.8s163.5 MiB [] 42% 0.8s163.5 MiB [] 43% 0.8s163.5 MiB [] 44% 0.8s163.5 MiB [] 45% 0.8s163.5 MiB [] 46% 0.8s163.5 MiB [] 47% 0.8s163.5 MiB [] 48% 0.7s163.5 MiB [] 49% 0.7s163.5 MiB [] 50% 0.7s163.5 MiB [] 51% 0.8s163.5 MiB [] 52% 0.7s163.5 MiB [] 54% 0.7s163

In [ ]:
import torch
import time
import asyncio
import os
import nest_asyncio
from transformers import AutoProcessor, AutoModelForVision2Seq, Blip2Processor
import gradio as gr
from playwright.async_api import async_playwright
from PIL import Image

nest_asyncio.apply()  # Fixes event loop issue in Colab

def log_to_file(message):
    """Logs messages for debugging."""
    with open("debug_log.txt", "a") as f:
        f.write(message + "\n")

class ProxyLiteVLMScraper:
    def __init__(self, url: str, save_screenshotpath: str):
        self.url = url
        self.image_save_path = save_screenshotpath
        self.browser = None
        self.page = None

    async def load_webpage(self):
        """Loads the website using Playwright."""
        playwright = await async_playwright().start()
        self.browser = await playwright.chromium.launch(headless=True)
        self.page = await self.browser.new_page()
        await self.page.goto(self.url)

    async def take_screenshot(self):
        """Takes a screenshot of the webpage."""
        await self.page.set_viewport_size({'width': 1920, 'height': 1080})
        await self.page.screenshot(path=self.image_save_path, full_page=True)

        # Ensure the screenshot is saved correctly
        if not os.path.exists(self.image_save_path):
            log_to_file("Error: Screenshot was not saved correctly.")
            return "Error: Screenshot could not be saved."

        return self.image_save_path

    async def run(self):
        await self.load_webpage()
        screenshot_path = await self.take_screenshot()
        return screenshot_path

def generate_response(image_path: str, user_query: str) -> str:
    """Generates response using Proxy-Lite-3B as a Vision-Language Model."""
    try:
        # Ensure the image is properly loaded
        image = Image.open(image_path).convert("RGB")

        if user_query is None or user_query.strip() == "":
            return "Error: Please provide a valid query."

        # Convert image and query into tensors
        inputs = processor(images=image, text=[user_query], return_tensors="pt").to(device)

        # Debugging: Log input tensor details
        log_to_file(f"Processor Inputs: {inputs}")

        if "pixel_values" not in inputs or inputs["pixel_values"] is None:
            return "Error: Image processing failed. Ensure the screenshot is correctly captured."

        with torch.no_grad():
            generated_ids = model.generate(
                pixel_values=inputs["pixel_values"],  # Explicitly pass pixel_values
                input_ids=inputs.get("input_ids", None),  # Ensure input_ids is optional
                max_new_tokens=250,
                num_beams=5,
                repetition_penalty=2.0,
                length_penalty=1.0,
                early_stopping=True
            )

        decoded_output = processor.batch_decode(generated_ids, skip_special_tokens=True)

        # Check if decoded_output is not None and is a list before proceeding
        if decoded_output and isinstance(decoded_output, list) and decoded_output[0]:
            return decoded_output[0]
        else:
            return "Error: Model could not generate a response."

    except Exception as e:
        log_to_file(f"Error generating response: {str(e)}")
        return f"Error generating response: {str(e)}"

def gradio_ui(url: str, query: str):
    start_time = time.time()
    scraper = ProxyLiteVLMScraper(url, save_screenshotpath="screenshot.png")

    loop = asyncio.get_event_loop()
    screenshot_path = loop.run_until_complete(scraper.run())

    log_to_file(f"Screenshot saved at: {screenshot_path}")
    response = generate_response(screenshot_path, query)
    total_time = time.time() - start_time
    log_to_file(f"Total Time Taken: {total_time:.2f} sec")
    return response

# Load Proxy-Lite-3B model
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "convergence-ai/proxy-lite-3b"
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b", trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto", trust_remote_code=True)

iface = gr.Interface(
    fn=gradio_ui,
    inputs=[
        gr.Textbox(label="Enter Website URL"),
        gr.Textbox(label="Enter Your Query"),
    ],
    outputs=gr.Textbox(label="Generated Response"),
    title="Proxy-Lite-3B VLM Web Scraper",
    description="Enter a website URL and a query, and the AI will take a screenshot and generate an answer."
)

iface.launch(share=True, debug=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Exception ignored in: <function BaseSubprocessTransport.__del__ at 0x79ac2fd03600>
Traceback (most recent call last):
  File "/usr/lib/python3.11/asyncio/base_subprocess.py", line 126, in __del__
    self.close()
  File "/usr/lib/python3.11/asyncio/base_subprocess.py", line 104, in close
    proto.pipe.close()
  File "/usr/lib/python3.11/asyncio/unix_events.py", line 765, in close
    self.write_eof()
  File "/usr/lib/python3.11/asyncio/unix_events.py", line 751, in write_eof
    self._loop.call_soon(self._call_connection_lost, None)
  File "/usr/lib/python3.11/asyncio/base_events.py", line 762, in call_soon
    self._check_closed()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 520, in _check_closed
    raise RuntimeError('Event loop is closed')
RuntimeError: Event loop is closed
ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-47' coro=<Connection.run() running at /usr/local/lib/python3.11/dist-packages/playwright/_impl/_connection.py:281>

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f3b04ebb3b9825ebb6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f3b04ebb3b9825ebb6.gradio.live
